In [1]:
import pandas as pd
import os

# specify the folder path where your Excel files are located
folder_path = "Z:\Analytics\Ani's Flow Analysis\\1st Step Lists"

# create an empty list to store the dataframes
dataframes = []

# loop through all the files in the folder
for filename in os.listdir(folder_path):
    # check if the file is an Excel file
    if filename.endswith('.xlsx'):
        # read the Excel file into a dataframe
        file_path = os.path.join(folder_path, filename)
        df = pd.read_excel(file_path)
        # add the dataframe to the list
        dataframes.append(df)

# concatenate all the dataframes into one dataframe
result = pd.concat(dataframes, axis=0, ignore_index=True)
print(result)

       CasinoID  Dep_amount_1  Bet_amount_1  Bonus_amount_1  Adj_bonus_1  \
0     101255989           0.0           0.0             0.0            0   
1     101255992           0.0           0.0             0.0            0   
2     101255993           0.0           0.0             0.0            0   
3     101255999           0.0           0.0             0.0            0   
4     101256000           0.0           0.0             0.0            0   
...         ...           ...           ...             ...          ...   
1627  101255980           0.0           0.0             0.0            0   
1628  101255982           0.0           0.0             0.0            0   
1629  101255984           0.0           0.0             0.0            0   
1630     194043           0.0           0.0             0.0            0   
1631     136395           0.0           0.0             0.0            0   

      Dep_amount_3  Bet_amount_3  Bonus_amount_3  Adj_bonus_3  Dep_amount_5  \
0       

In [2]:
users = result['CasinoID']

In [3]:
import pyodbc

cnx = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DWH;"
    "Database=dwOper;"
    "Trusted_Connection=yes;")

query = """
SELECT u.Base_UserID                                  AS CasinoID,
       SUM(p.Amount)                                  AS Amount
FROM Payment AS p
INNER JOIN VIEW_PlatformPartnerUsers_TotogamingAm AS u ON u.UserID = p.UserID
WHERE p.PaymentTypeID = 2
    AND p.PaymentStatusID = 8
    AND CAST(DATEADD(hour, -4, p.modify_date) AS DATE) >= '2023-01-01'
    AND CAST(DATEADD(hour, -4, p.modify_date) AS DATE) < CAST(GETDATE() AS DATE)
    AND u.PartnerID IN (237)
    AND p.SourceID = 2
GROUP BY u.Base_UserID
"""

In [4]:
deposit_data = pd.read_sql(query, cnx)
cnx.close()

C:\Users\narek.meloyan\AppData\Local\Temp\ipykernel_38608\2081995878.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  deposit_data = pd.read_sql(query, cnx)


In [5]:
deposit_data.head()

,CasinoID,Amount
0,820781,8500.0
1,287546,28600.0
2,180449,1164620.0
3,707666,1132140.0
4,991202,16000.0


In [6]:
import numpy as np
Q1 = deposit_data['Amount'].quantile(0.25)
Q3 = deposit_data['Amount'].quantile(0.75)
IQR = Q3 - Q1

thresh = 1.5 * IQR

deposit_data['Outlier'] = np.where((deposit_data['Amount'] < (Q1 - thresh)) | (deposit_data['Amount'] > (Q3 + thresh)), True, False)

In [7]:
deposit_data

,CasinoID,Amount,Outlier
0,820781,8500.0,False
1,287546,28600.0,False
2,180449,1164620.0,True
3,707666,1132140.0,True
4,991202,16000.0,False
...,...,...,...
99917,528589,1000.0,False
99918,869524,1000.0,False
99919,828363,1000.0,False
99920,1026325,5000.0,False


In [9]:
users = users.to_frame().merge(deposit_data, on='CasinoID')

In [11]:
users[users['Outlier'] == True]

,CasinoID,Amount,Outlier
42,101256685,646850.0,True
129,101253507,2392020.0,True
136,101253581,4328100.0,True
178,101254123,303290.0,True
191,101254242,623100.0,True
192,101254253,357000.0,True
233,101254509,280550.0,True
247,101254613,1501000.0,True
264,101254683,200000.0,True
283,101254751,176000.0,True
